# FinGraph Demo

welcome to **FinGraph**, a financial knowledge graph project for analyzing systemic risk and contagion in financial networks

in this demo, you will:

- load company and supply-chain data  
- build a network graph representing companies and their relationships  
- compute centrality metrics and detect communities  
- simulate financial contagion through the network  
- visualize the graph interactively with risk-colored nodes

In [ ]:
!pip install pandas numpy networkx pyvis python-louvain matplotlib seaborn scipy etl

You should consider upgrading via the 'C:\Users\youce\Desktop\FinGraph\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
import importlib.util
import sys
from pathlib import Path

src_path = Path('../src/etl.py').resolve()

spec = importlib.util.spec_from_file_location("etl_local", src_path)
etl_local = importlib.util.module_from_spec(spec)
sys.modules["etl_local"] = etl_local
spec.loader.exec_module(etl_local)

companies, edges = etl_local.load_data('../data/companies.csv', '../data/supply_edges.csv')
G = etl_local.build_graph(companies, edges)

In [ ]:
import sys
sys.path.append('../src')  

from analytics import compute_centrality, detect_communities, attach_metrics
from contagion import simulate_contagion


In [26]:
import sys
from pathlib import Path
import importlib.util

In [19]:
import pandas as pd
import numpy as np
import networkx as nx

In [20]:
sys.path.append('../src')  

In [28]:
src_path = Path('../src/etl.py').resolve()
spec = importlib.util.spec_from_file_location("etl_local", src_path)
etl_local = importlib.util.module_from_spec(spec)
sys.modules["etl_local"] = etl_local
spec.loader.exec_module(etl_local)

In [31]:
from viz import visualize_graph

In [32]:
companies, edges = etl_local.load_data('../data/companies.csv', '../data/supply_edges.csv')

G = etl_local.build_graph(companies, edges)

print(f"✅ Graph built with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

✅ Graph built with 6 nodes and 7 edges.


the nodes represent the companies and the edges represent the connections between them

In [33]:
centrality = compute_centrality(G)
communities = detect_communities(G)

G = attach_metrics(G, centrality, communities)

print("🏆 Top nodes by PageRank:")
for node, score in sorted(centrality['pagerank'].items(), key=lambda x: x[1], reverse=True):
    print(f"{node} ({G.nodes[node]['name']}): {score:.4f}")

print("\n🌐 Communities detected:")
for cid, members in communities.items():
    print(f"Community {cid}: {members}")

🏆 Top nodes by PageRank:
AAPL (Apple Inc.): 0.3171
TSLA (Tesla): 0.3043
MSFT (Microsoft): 0.3036
JPM (JPMorgan Chase): 0.0250
BABA (Alibaba Group): 0.0250
TSM (TSMC): 0.0250

🌐 Communities detected:
Community AAPL: 0
Community MSFT: 0
Community TSLA: 1
Community JPM: 1
Community BABA: 0
Community TSM: 0


now that are our financial knowledge graph is built, we can analyze it using **graph algorithms**:

- **centrality:**
  it helps us identify the most "influential" or systemically important companies in the network
   - *pagerank*: measures overall importance in the network
   - *betweenness*: highlights companies acting as bridges between clusters
   - *degree*: simply counts direct connections

- **community detection:**
  groups companies into clusters based on their network structure (eg; tech firms together, banks together)
  this helps us see sector-level or regional structures that might be hidden in raw data

In [36]:
G = simulate_contagion(G, initial_nodes=['TSLA', 'JPM'], steps=5, decay=0.5)

risk_sorted = sorted(G.nodes(data=True), key=lambda x: x[1].get('risk', 0), reverse=True)
print("Top nodes by simulated risk:")
for node, data in risk_sorted:
    print(f"{node} ({data['name']}): {data.get('risk', 0):.3f}")


Top nodes by simulated risk:
MSFT (Microsoft): 1.000
TSLA (Tesla): 1.000
JPM (JPMorgan Chase): 1.000
AAPL (Apple Inc.): 0.500
BABA (Alibaba Group): 0.000
TSM (TSMC): 0.000


now our graph is here, let's break things down:

the simulated risk scores were calculated using a **contagion model** on the financial network:

1. **initialization:**

selected seed nodes (i selected `TSLA` and `JPM`) start with a risk score of **1.0**

2. **propagation:**

risk spreads from each node to its direct neighbors along supply-chain connections
at each step, the risk decays by a factor (`decay=0.5`) so downstream nodes get lower exposure

3. **final scores:**

- nodes directly connected to highly risky companies inherit some risk

- nodes further away or unconnected remain at **0 risk**


each node in the financial network is colored based on its **simulated risk score** from the contagion model simulation

- **red nodes** --> `risk>0.66`; high risk, likely to prpagate failures
- **yellow nodes** --> `0.33<risk<0.66`; moderate risk, some exposure to contagion
- **green nodes** --> `risk<0.33`; low risk, relatively safe in the network

this color coding allows you to **quickly identify systemically important companies** and visualize how financial shocks spread across sectors and supply chains

In [39]:
visualize_graph(G)

🔥 Interactive graph saved to graph.html!


## ✅ Summary & Insights

in this notebook, i built **FinGraph**, a financial knowledge graph, and analyzed systemic risk and contagion across companies:

1. **graph construction:**  
   companies and supply-chain relationships were represented as nodes and edges

2. **network analysis:**  
   centrality metrics identified systemically important companies, and community detection revealed clusters of related firms

3. **contagion simulation:**  
   simulated financial shocks propagated through the network from seed nodes (`TSLA` and `JPM`), producing **risk scores** for each company

4. **visualization:**  
   an interactive graph shows node risk via a **green → yellow → red** gradient, making it easy to identify vulnerable companies and hidden dependencies

💡 **key takeaways:**  
- `MSFT`, `TSLA`, and `JPM` are the highest-risk nodes in this network  
- nodes like `AAPL` inherit moderate risk through their connections
- companies with no direct exposure remain low-risk  

this workflow demonstrates how **graph-based financial modeling** can provide actionable insights beyond traditional spreadsheet analysis, helping analysts detect systemic risk and sector contagion early